# Пример использования сценария

## Загрузка данных <a name="data-loader"></a>

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from sponge_bob_magic.sber_datasets import SberAdviser

In [4]:
sber_adviser = SberAdviser("../../SberAdviser/data", only_positive=False)

/home/shminke-ba/projects/sponge-bob-magic/venv/lib/python3.6/site-packages/pyarrow/util.py:39: FutureWarning: pyarrow.open_stream is deprecated as of 0.17.0, please use pyarrow.ipc.open_stream instead
  warnings.warn(msg, FutureWarning)


In [5]:
# положительным примером взаимодействия считаются сделки больше определённой суммы
# user_id --- тот, кому подыскивается покупателей, item_id --- потенциальный покупатель
sber_adviser.log

DataFrame[user_id: string, item_id: string, relevance: int]

In [6]:
# в общей сложности в базе имеется более 10 млн ИНН
# но для пилота отбираются только те, которые работают в Нижегородской области
print(sber_adviser.log.count())
print(sber_adviser.log.select("user_id").distinct().count())
print(sber_adviser.log.select("item_id").distinct().count())

4092586
8460
40422


In [7]:
sber_adviser.log.filter("relevance == 1").count() / sber_adviser.log.count()

0.017654607624616805

In [8]:
# поскольку задача симметрика (user и item --- это компании)
# то свойства пользователей и объектов --- это одно и то же
print(sber_adviser.user_features.count())
print(sber_adviser.item_features.count())

41071
41071


In [9]:
# на данный момент большинство свойств заполнено нулями
# поскольку источники этих свойств --- тематическая модель и аналогичные
sber_adviser.user_features

DataFrame[user_id: string, wntm_0: double, wntm_1: double, wntm_2: double, wntm_3: double, wntm_4: double, wntm_5: double, wntm_6: double, wntm_7: double, wntm_8: double, wntm_9: double, wntm_10: double, wntm_11: double, wntm_12: double, wntm_13: double, wntm_14: double, wntm_15: double, wntm_16: double, wntm_17: double, wntm_18: double, wntm_19: double, wntm_20: double, wntm_21: double, wntm_22: double, wntm_23: double, wntm_24: double, wntm_25: double, wntm_26: double, wntm_27: double, wntm_28: double, wntm_29: double, wntm_30: double, wntm_31: double, wntm_32: double, wntm_33: double, wntm_34: double, wntm_35: double, wntm_36: double, wntm_37: double, wntm_38: double, wntm_39: double, wntm_40: double, wntm_41: double, wntm_42: double, wntm_43: double, wntm_44: double, wntm_45: double, wntm_46: double, wntm_47: double, wntm_48: double, wntm_49: double, wntm_50: double, wntm_51: double, wntm_52: double, wntm_53: double, wntm_54: double, wntm_55: double, wntm_56: double, wntm_57: doubl

## Сценарий с разными моделями

In [10]:
from sponge_bob_magic.models import SLIM, ClassifierRec

classifier = ClassifierRec()
slim = SLIM()

##  Схема валидации

In [19]:
from sponge_bob_magic.splitters import ColdUserRandomSplitter, RandomSplitter

random_splitter = RandomSplitter(
    test_size=0.3,
    drop_cold_items=True,
    drop_cold_users=True,
    seed=1234
)
cold_user_splitter = ColdUserRandomSplitter(0.3)

### Обучение сценария <a name="fit-scenario"></a>

In [12]:
from sponge_bob_magic.scenarios import MainScenario
from sponge_bob_magic.metrics import NDCG, HitRate

scenario = MainScenario(
    splitter=user_random_splitter,
    recommender=slim,
    criterion=HitRate,
    metrics={
        NDCG: [10],
        HitRate: [10],
    }
)

In [13]:
# использовать трансформ из либы
active_users = (
    sber_adviser.log.select("user_id", "relevance")
    .groupby("user_id").sum().filter("sum(relevance) > 10").select("user_id")
)
log = sber_adviser.log.join(active_users, on="user_id", how="inner")

In [14]:
print(log.filter("relevance = 1").count())
print(log.filter("relevance = 1").select("user_id").distinct().count())
print(log.filter("relevance = 1").select("item_id").distinct().count())

52642
1490
17267


In [15]:
scenario.research(
    {
        "beta": [2 ** i for i in range(-8, -4)],
        "lambda_": [2 ** i for i in range(-8, -4)],
    },
    log.filter("relevance = 1"),
    k=10,
    n_trials=1
)
scenario.experiment.results

02-Jul-20 11:41:27, sponge_bob_magic, DEBUG: Деление лога на обучающую и тестовую выборку
02-Jul-20 11:41:30, sponge_bob_magic, DEBUG: Длина трейна и теста: 35922 16720
02-Jul-20 11:41:30, sponge_bob_magic, DEBUG: Количество пользователей в трейне и тесте: 1038, 452
02-Jul-20 11:41:31, sponge_bob_magic, DEBUG: Количество объектов в трейне и тесте: 14037, 9220
02-Jul-20 11:41:31, sponge_bob_magic, DEBUG: Инициализация метрик
02-Jul-20 11:41:31, sponge_bob_magic, DEBUG: Обучение и предсказание дополнительной модели
02-Jul-20 11:41:31, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
02-Jul-20 11:41:32, sponge_bob_magic, DEBUG: Основная стадия обучения (fit)
02-Jul-20 11:41:35, sponge_bob_magic, DEBUG: Список item содержит элементы, которые отсутствовали при обучении. Результат предсказания будет не полным.
/home/shminke-ba/projects/sponge-bob-magic/venv/lib/python3.6/site-packages/pyarrow/util.py:39: FutureWarning: pyarrow.open_stream is deprecated as of 0.17.0, please 

,HitRate@10,NDCG@10
"SLIM{'beta': 0.00390625, 'lambda_': 0.00390625}",0.227876,0.04333


In [18]:
scenario.recommender = classifier
scenario.research(
    {
        "n_trees": [1],
    },
    log,
    k=10,
    n_trials=1,
    user_features=sber_adviser.user_features,
    item_features=sber_adviser.item_features
)
scenario.experiment.results

02-Jul-20 11:49:12, sponge_bob_magic, DEBUG: Деление лога на обучающую и тестовую выборку
02-Jul-20 11:49:16, sponge_bob_magic, DEBUG: Длина трейна и теста: 1157486 16858
02-Jul-20 11:49:16, sponge_bob_magic, DEBUG: Количество пользователей в трейне и тесте: 1043, 447
02-Jul-20 11:49:17, sponge_bob_magic, DEBUG: Количество объектов в трейне и тесте: 36227, 8960
02-Jul-20 11:49:17, sponge_bob_magic, DEBUG: Инициализация метрик
02-Jul-20 11:49:17, sponge_bob_magic, DEBUG: Обучение и предсказание дополнительной модели
02-Jul-20 11:49:17, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
02-Jul-20 11:49:19, sponge_bob_magic, DEBUG: Основная стадия обучения (fit)
02-Jul-20 11:49:28, sponge_bob_magic, DEBUG: Список item содержит элементы, которые отсутствовали при обучении. Результат предсказания будет не полным.
/home/shminke-ba/projects/sponge-bob-magic/venv/lib/python3.6/site-packages/pyarrow/util.py:39: FutureWarning: pyarrow.open_stream is deprecated as of 0.17.0, pleas

,HitRate@10,NDCG@10
ClassifierRec{'n_trees': 1},0.087248,0.012288
